In [1]:
import simplejson as json
parsed_corpus = open('parsed_corpus.txt', encoding='utf-8')

pos_left = dict()
pos_right = dict()

In [2]:
def process_sentence(stc):
    tags = list()
    for l in stc:
        tags.extend([p['tag'] for p in l['pos']])
        
    for i, t in enumerate(tags):
        if t == 'ZST':
            left = None
            right = right = tags[i + 1]
        elif t == 'ZED':
            left = left = tags[i - 1]
            right = None
        else:
            left = tags[i - 1]
            right = tags[i + 1]
        
        if t not in pos_left:
            pos_left[t] = dict()
        if t not in pos_right:
            pos_right[t] = dict()
        
        if left is not None:
            pos_left[t][left] = pos_left[t][left] + 1 if left in pos_left[t] else 1
        if right is not None:
            pos_right[t][right] = pos_right[t][right] + 1 if right in pos_right[t] else 1

In [3]:
from tqdm import tqdm

line = parsed_corpus.readline()
count = 0
while line != '' and line != '\n':
    process_sentence(json.loads(line))
    del line
    line = parsed_corpus.readline()
    count += 1
    if count % 10000 == 0:
        print(count)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000


In [4]:
import sqlite3

db = sqlite3.connect('dictionary.db')
c = db.cursor()
tokens = dict()

c.execute('CREATE TABLE `left_pos` ( `pos` TEXT, `target` TEXT, `count` INTEGER )')
c.execute('CREATE TABLE `right_pos` ( `pos` TEXT, `target` TEXT, `count` INTEGER )')


In [5]:
for pos in pos_left:
    for target in pos_left[pos]:
        count = pos_left[pos][target]
        c.execute('INSERT INTO left_pos VALUES (?, ?, ?)', (pos, target, count))

for pos in pos_right:
    for target in pos_right[pos]:
        count = pos_right[pos][target]
        c.execute('INSERT INTO right_pos VALUES (?, ?, ?)', (pos, target, count))
        
db.commit()
db.close()